In [39]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions

import numpy as np
import tensorflow as tf

pipeline_options = PipelineOptions()
pipeline_options.view_as(SetupOptions).save_main_session = True

In [ ]:
p = beam.Pipeline(options=pipeline_options)

infile = '../notes/wordcount-example.md'
outfile = '../notes/wordcount-example-OUT.md'

lines = p | beam.io.ReadFromText(infile)

out_lines = lines | beam.io.WriteToText(outfile)

p.run()

# tfx

In [2]:
from tfx.utils import types

In [3]:
import collections

In [4]:
isinstance([], collections.Iterable)

True

# MNIST - read byte files

In [16]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels
from tensorflow.python.platform import gfile

In [17]:
# extract_images

In [18]:
local_file = '/tmp/data/train-images-idx3-ubyte.gz'
with gfile.Open(local_file, 'rb') as f:
    train_images = extract_images(f)

Extracting /tmp/data/train-images-idx3-ubyte.gz


In [19]:
type(train_images), train_images.shape

(numpy.ndarray, (60000, 28, 28, 1))

In [29]:
local_file = '/tmp/data/train-labels-idx1-ubyte.gz'
with gfile.Open(local_file, 'rb') as f:
    train_labels = extract_labels(f)

Extracting /tmp/data/train-labels-idx1-ubyte.gz


In [30]:
type(train_labels), train_labels.shape

(numpy.ndarray, (60000,))

In [31]:
rows = train_images.shape[1]
cols = train_images.shape[2]
depth = train_images.shape[3]

rows, cols, depth

(28, 28, 1)

In [33]:
index = 0
label = train_labels[index]
type(label), label

(numpy.uint8, 5)

In [34]:
image = train_images[0]
type(image)

numpy.ndarray

In [47]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _dict_to_example(data):
    feature = {}
    for k,v in data.items():
        if isinstance(v, np.ndarray):
            feature[k] = _bytes_feature(v.tostring())
        elif isinstance(v, np.uint8):
            feature[k] = _int64_feature(int(v))
        elif isinstance(v, int):
            feature[k] = _int64_feature(v)
    return tf.train.Example(features=tf.train.Features(feature=feature))

data = {
  'height': rows,
  'width': cols,
  'depth': depth,
  'label': label,
  'image_raw': image
}

example = _dict_to_example(data)

example

features {
  feature {
    key: "depth"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "height"
    value {
      int64_list {
        value: 28
      }
    }
  }
  feature {
    key: "image_raw"
    value {
      bytes_list {
        value: "\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\003\022\022\022~\210\257\032\246\377\367\177\000\000\000\000\000\000\000\000\000\000\000\000\036$^\232\252\3